<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 18: Numerical ODE Simulations Part II</h1>


<a name='section_18_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L18.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_18_1">L18.1 Bifurcation Diagrams and Chaotic Motion</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_18_1">L18.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_18_2">L18.2 Numerical Precision and Chaotic Dynamics</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_18_2">L18.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_18_3">L18.3 Machine Learning the Pendulum Part I</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_18_3">L18.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_18_4">L18.4 Machine Learning the Pendulum Part II</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_18_4">L18.4 Exercises</a></td>
    </tr>
</table>

<h3>Learning Objectives</h3>

In this lesson, we will build upon our knowledge of numerical integrators to investigate systems that exhibit chaotic motion. We will then return to the example of a simple pendulum and apply machine learning tools to our simulations. Finally, we will solve a more complicated example: simulating the density of relic dark matter in the universe.

<h3>Slides</h3>

You can access the slides related to this lecture at the following link: <a href="https://github.com/mitx-8s50/slides/raw/main/module3_slides/L18_slides.pdf" target="_blank">L18 Slides</a>

<h3>Data</h3>

Download the directory where we will save data.

In [ ]:
#>>>RUN: L18.0-runcell00

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L18' >> .git/info/sparse-checkout
!git pull origin main

#NOTE: must create L18 directory on github

<h3>Installing Tools</h3>

Before we do anything, let's make sure we install the tools we need.

In [ ]:
#>>>RUN: L18.0-runcell01

#you may need to install this package (it already available in Colab)
#!pip install imageio    #https://imageio.readthedocs.io/en/stable/

<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 

In [ ]:
#>>>RUN: L18.0-runcell02

import imageio                        #https://imageio.readthedocs.io/en/stable/
from PIL import Image                 #https://pillow.readthedocs.io/en/stable/reference/Image.html

import numpy as np                    #https://numpy.org/doc/stable/
import torch                          #https://pytorch.org/docs/stable/torch.html
import torch.nn as nn                 #https://pytorch.org/docs/stable/nn.html
import matplotlib.pyplot as plt       #https://matplotlib.org/stable/api/pyplot_summary.html#module-matplotlib.pyplot

from scipy.integrate import odeint    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
from scipy.optimize import minimize   #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
import csv                            #https://docs.python.org/3/library/csv.html
from matplotlib.patches import Circle #https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Circle.html
from scipy.integrate import solve_ivp #https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html
from IPython.display import Image     #https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: L18.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_18_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L18.1 Bifurcation Diagrams and Chaotic Motion</h2>  

| [Top](#section_18_0) | [Previous Section](#section_18_0) | [Exercises](#exercises_18_1) | [Next Section](#section_18_2) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS18/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS18_vid1" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

Now, what we can do is define a bifurcation diagram using a driven damped harmonic oscillator. Bifurcation dynamics are used to describe chaotic motion like what we saw with the damped, driven pendulum.

Before we do that, let's create the most famous bifurcation diagram.  In this map, we perform an iterative procedure whereby we define

$$x_{n+1} = r x_{n}(1-x_{n})$$

Here, $x$ varies from 0 to 1. As you can see, when $x$ is small, its value increases by roughly a factor of $r$ at each iteration. However, when $x$ gets close to 1, the $(1-x_{n})$ begins to dominate and $x$ can start to go back down. An example of such a system is a population with a reproduction rate of $r$ but with limited resources so that when $x_{n}$ gets too large, the population can decline.

What we can then do is run the logistic regression and define a bifurcation diagram. In this diagram, we plot the $x$ position for the n-th iteration by varying $r$ over a large number of values after starting at the exact same initial $x$ position. Specifically, we start with a value of $x_0$ very close to zero, vary $r$ in 10,000 steps from 2.5 to 4.0, and plot the 200 points between $n$ of 1800 and 2000.

In [ ]:
#>>>RUN: L18.1-runcell01

def logistic(r, x):
    return r * x * (1 - x)

#vary parameters to change density of points
iterations=2000
last = 200
n = 10000
r = np.linspace(2.5, 4.0, n)
x = 1e-5 * np.ones(n)

fig, ax1 = plt.subplots(1, 1, figsize=(8, 9),sharex=True)
for i in range(iterations):
    x = logistic(r, x)   
    # We compute the partial sum of the
    # Lyapunov exponent.<a name='section_15_2'></a>
    #lyapunov += np.log(abs(r - 2 * r * x))
    # We display the bifurcation diagram.
    if i >= (iterations - last):
        ax1.plot(r, x, ',k', alpha=.25)
ax1.set_xlabel('r')
ax1.set_ylabel('x')
ax1.set_xlim(2.5, 4)
ax1.set_title("Bifurcation diagram")


It's surprising that such  a simple formula, like the completely deterministic one simulated above, can produce such bizarre looking results. The name "bifurcation diagram" comes from the points where the ending position suddenly starts flipping between two different values. Then, there are ranges of $r$ where the system seems to go crazy, with final positions all over the place. There are other oddities, like darker lines marking final points that occur more frequently and weird gaps where the system never appears. Try running more iterations and/or plotting more points and/or scanning $r$ in finer steps to convince yourself that the system never finds itself in these "disallowed" regions.

Systems where the final state can vary wildly over a large range are called "chaotic". Even though the evolution is totally deterministic, it becomes impossible to predict the final outcome since infinitesimal changes in the conditions can give   completely different results. The study of examples of chaotic systems has grown into a very robust area of research in many fields.

Let's try to do the same thing with our numerically solved damped driven pendulum. Let's start by making our pendulum a bit chaotic, here are some variables that give us what looks like chaotic motion. Let's go ahead and see how it looks. Note the much stronger driving force (the $fp$ parameter) compared to what was used in the previous Lesson.

In [ ]:
#>>>RUN: L18.1-runcell02

def df(t, thetavec, g, l, mup, fp, omega):
    theta, thetadot = thetavec
    return [thetadot, -(g/l) * np.sin(theta) - mup*thetadot + fp*np.cos(omega*t) ]

g=9.8
l=9.8
thetainit=0.2
mu=0.5
fp=1.2
omega=2./3.
solution = solve_ivp(df, [0., 100], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fp,omega))

plt.plot(solution.t, solution.y[0] % (2*np.pi),label="int-position")
plt.plot(solution.t, solution.y[1],label="int-velocity")
plt.xlabel("time(s)")
plt.ylabel("position")
plt.legend()
plt.show()


This result looks nothing like the smoothly oscillating pendulum that we saw previously. The strong driving force causes lots of strange jumps. Now, one thing that is interesting is to look at the variation of the oscillations over a period. The way we do this is by looking at the so-called "stable" points in the oscillation, namely those where the velocity is zero. We will solve the pendulum for 200 different values of the driving force.

In [ ]:
#>>>RUN: L18.1-runcell03

#NOTE: Running this code will take several minutes
#You can ignore the warning messages about redundant colors.

fp=1.0
thetainit=0.2
mu=0.5
g=9.8
l=9.8
omegad=2./3.
        
for scale in np.arange(1.0,1.2,0.001):
    fptrue=fp*scale
    solution = solve_ivp(df, [0., 1000], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fptrue,omegad))
    crossings = np.where((solution.y[1][-1000:][:-1]*solution.y[1][-1000:][1:]  <= 0)
                         & (solution.y[1][-1000:][:-1] - solution.y[1][-1000:][1:] > 0)
                        )[0]
    plt.plot(fptrue*np.ones(len(crossings)), (solution.y[0][-1000:][crossings] % (2*np.pi)),',k',c='black',marker=".",markersize=2)#, alpha=.25)

plt.xlabel("Driving Force Amplitude (fp)")
plt.ylabel("Phase at Zero-Crossings of Velocity")
plt.show()

What's going on? We are plotting the positions where the pendulum changes direction. You can clearly see that there are 2 or 4 well defined stopping points at some values of `fp`. Then, there are `fp` ranges where the number of stable points changes pretty dramatically and, at the larger values, there is no clearly defined stable point at all. This is emblematic of driven pendulum dynamics.

In the real world, this type of chaotic motion is present in a lot of different physical scenarios. One classic example is that of Saturn's rings, which themselves are a damped driven harmonic oscillator if you take a slice of the dust orbiting around Saturn. The flow of the dust pulls the other dust, and Saturn's gravity pulls the dust closer. The moons of Saturn also have a significant impact on the ring structure.

Using what we have already done, we can create a full simulation of these rings without much work. Out of this, you can demonstrate the clumpy features and divisions of Saturn's rings, such as the Cassini division. The field of chaotic dynamics is rich, but requires highly adept integrators that can extrapolate over large regions. This is a critical component of understanding the dynamics of predicting future motion of a system.

<a name='exercises_18_1'></a>     

| [Top](#section_18_0) | [Restart Section](#section_18_1) | [Next Section](#section_18_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 18.1.1</span>

In a bifurcation diagram, the $x$ axis represents the values of the bifurcation parameter (generically called $r$ in this exercise), while the $y$ axis represents the values  of some property of the system. The diagram shows how the behavior of the system evolves as $r$ changes.

Features of a bifurcation diagram can include the following:

**Fixed Points and Stability:** If, for a particular $r$, the system converges to a stable value, it will be represented as a single point in the bifurcation diagram.

**Periodic Behavior:** The appearance of multiple points or regions in the bifurcation diagram is indicative of periodic behavior in the system. As $r$ varies, the system may transition from stable fixed points to periodic orbits and then to chaos.

**Bifurcation Points:** Bifurcation points are locations in the diagram where a qualitative change in the system's behavior occurs. This can manifest as a branching or splitting of trajectories.

**Period-Doubling Cascades:** Successive bifurcation points, leading to the doubling of the period of the system's behavior, are characteristic structures known as period-doubling cascades.

**Chaos:** Chaotic behavior is often observed in regions of the bifurcation diagram where the system exhibits an extremely sensitive dependence on the initial conditions.


Examine the bifurcation diagram below. Which item from the list below best characterizes the feature indicated by the arrow?

A) Fixed Point

B) Periodic Behavior

C) Bifurcation Point

D) Period-Doubling Cascade

E) Chaos

<p align="center">
<img alt="Chaotic Phase Space Plot" src="https://raw.githubusercontent.com/mitx-8s50/images/main/L18/chaotic_phase_space.png" width="600"/>
</p>

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 18.1.2</span>

Now, take a more detailed look at the state of the system in the bifurcation diagram for the pendulum example, generated by code cell `L18.1-runcell03`. Make plots of position $\theta$ on the $x$ axis vs. velocity $\dot{\theta}$ on the $y$ axis, for two different values of `fp`, as follows: (1) `fp` corresponding to a stable point, and (2) a `fp` corresponding to a chaotic region. What behavior do you observe in the last 5000 steps of the simulation (after initial variation has subsided)?


A) Both plots follow repeating trajectories.\
B) One plot repeats itself, while the other fills a large fraction of the full phase space.\
C) Both plots fill a large fraction of the full phase space.


In [ ]:
#>>>EXERCISE: L18.1.2

#plot 1: stable point
scale_stable = #YOUR CODE HERE (choose value based on output of L18.1-runcell03)

solution = solve_ivp(df, [0., 100000*0.01], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fp*scale_stable,omegad))
plt.plot((solution.y[0,-5000:] + np.pi) % (2*np.pi), solution.y[1,-5000:] ,label="int-position")
plt.xlabel("position")
plt.ylabel("velocity")
plt.title('Stable Dynamics')
plt.show()

#plot 2: chaotic region
scale_chaotic = #YOUR CODE HERE (choose value based on output of L18.1-runcell03)

solution = solve_ivp(df, [0., 100000*0.01], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fp*scale_chaotic,omegad))
plt.plot((solution.y[0,-5000:] + np.pi) % (2*np.pi), solution.y[1,-5000:] ,label="int-position")
plt.xlabel("position")
plt.ylabel("velocity")
plt.title('Chaotic Dynamics')
plt.show()


<a name='section_18_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L18.2 Numerical Precision and Chaotic Dynamics</h2>  

| [Top](#section_18_0) | [Previous Section](#section_18_1) | [Exercises](#exercises_18_2) | [Next Section](#section_18_3) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS18/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS18_vid2" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

Now, one additional thing we can do is look at the performance with some of our simpler integration methods. Let's do a quick comparison of the Poincare section plots $\dot{\theta}$ vs $\theta$ for different levels of precision.

For this, what we will do is run the leap-frog integrator on the same system, and we will try to gauge our overall performance compared to the IVP. We can do this comparison for both a stable and an unstable scenario. Let's see what we get.

In [ ]:
#>>>RUN: L18.2-runcell01

def store(theta,thetapos,time,timepos):
    thetapos = np.vstack((thetapos,theta))
    timepos  = np.vstack((timepos,time))
    return timepos,thetapos

def step(theta,t,dt,g,l,mu,f,omegad):
    theta[0] += theta[1] * dt 
    #theta[1] += (g/l)*(-1)*theta[0]*dt
    #print(((g/l)*(-1)*np.sin(theta[0])-mu*theta[1]+f*np.sin(omegad*t))*dt)
    theta[1] += ((g/l)*(-1)*np.sin(theta[0])-mu*theta[1]+f*np.sin(omegad*t))*dt
   
def loop(nsteps=1000,dt=0.01,g=9.8,l=9.8,mu=0.5,f=1.0,omegad=omegad):
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = thetainit #initial conditions
    theta[1] = 0.0
    theta[1] += ((g/l)*(-1)*np.sin(theta[0])-mu*theta[1]+f*np.sin(omegad*time))*dt*0.5
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        step(theta,time,dt,g,l,mu,f,omegad)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos

time,thetaout=loop(100000,dt=0.01,g=g,l=l,mu=mu,f=fp*1.05,omegad=omegad)
print(thetaout)
solution = solve_ivp(df, [0., 100000*0.01], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fp*1.05,omegad))
plt.plot((solution.y[0] + np.pi) % (2*np.pi), solution.y[1] ,label="IVP")
plt.plot((thetaout[:,0] + np.pi) % (2*np.pi), thetaout[:,1] ,label="LEAP-FROG")
plt.xlabel("position")
plt.ylabel("velocity")
plt.legend()
plt.show()

time,thetaout=loop(100000,dt=0.01,g=g,l=l,mu=mu,f=fp*1.12,omegad=omegad)
print(thetaout)
solution = solve_ivp(df, [0., 100000*0.01], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fp*1.12,omegad))
plt.plot((solution.y[0] + np.pi) % (2*np.pi), solution.y[1] ,label="IVP")
plt.plot((thetaout[:,0] + np.pi) % (2*np.pi), thetaout[:,1] ,label="LEAP-FROG")
plt.xlabel("position")
plt.ylabel("velocity")
plt.legend()
plt.show()

What you can see is that the leap-frog integrator is not nearly as thorough as the IVP integrator, especially for the unstable case where the leap-frog seems to get stuck at certain spots and is unable to fill as much of the phase space. This doesn't come as a surprise since we know that the leap-frog integrator is only just an approximation of what can be achieved using the full modern integrators.

<a name='exercises_18_2'></a>     

| [Top](#section_18_0) | [Restart Section](#section_18_2) | [Next Section](#section_18_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 18.2.1</span>

Make a bifurcation diagram that is similar to the one produced by `L18.1-runcell03`, now using the Leap-Frog integrator (this will take a long time). Are the regions of stability, bifurcation, and choas the same? Choose an answer below:

A) No, the regions are entirely different and not at all similar.

B) Yes, the regions are identical.

C) The features are similar, but not identical.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 18.2.2</span>

The purpose of a Poincaré section is to simplify the analysis of complex, high-dimensional systems by examining their behavior in a lower-dimensional space that is meaningful.

From the following options, select ALL answers which are valid examples of Poincaré sections:

A) A plane perpendicular to the trajectory of a swinging pendulum, intersecting the trajectory at each oscillation.

B) A random slice through the three-dimensional phase space of a chaotic system, capturing points at arbitrary locations.

C) A sphere enclosing the orbit of a satellite around a planet, capturing points as the satellite completes each orbit.

D) A plane tangential to the trajectory of a spinning top, capturing points as the top precesses.


<a name='section_18_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L18.3 Machine Learning the Pendulum Part I</h2>  

| [Top](#section_18_0) | [Previous Section](#section_18_2) | [Exercises](#exercises_18_3) | [Next Section](#section_18_4) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS18/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS18_vid3" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Review of Deep Learning Methods</h3>

In the previous module of this course, we introduced deep learning methods, which utilize neural networks (NN) as powerful function approximators. Neural networks learn from data by adjusting their internal parameters to capture complex patterns and relationships within the data. This learning process involves minimizing a loss function, which quantifies the difference between the network's predictions and the actual outcomes. Essentially, a neural network can be viewed as a sophisticated fitting procedure that iteratively improves its accuracy by reducing the loss function, ultimately providing a model that can generalize well to new, unseen data.



Now that we have opened Pandora's box on deep learning, we can't go a lecture without discussing deep learning applications to everything.

Deep learning for differential equations also has a very "Physicsy" moniker and is often called "Physics Informed Machine Learning" (PIML) or sometimes "Physics Informed Neural Networks" (PINN). Ironically, the people who really pushed this are not physicists.

The idea to do Physics Informed Machine Learning stems from the ability of neural networks to be function approximators. The simplest form is to do the same function style "fitting" that is done in deep learning regression. However, what we can do is modify our loss with the knowledge of the differential equation.

To seed the idea, let's consider modifying the loss with an additional term given by our differential equation. For a damped harmonic oscillator we can write:

$$
F = -kx - \mu\dot{x} \\
m \ddot{x} = -kx - \mu\dot{x} \\
m \ddot{x} + mu\dot{x} +  kx  = 0
$$


Since the quantity in the last equation shown above is conserved, we can approximate the requirements of the differential equation by writing a loss term as:

$$
\mathcal{L_{\rm constrained}} = \left( m \ddot{x} + \mu\dot{x} +  kx \right)^2
$$

We square the quantity to ensure that it is positive, so this is just the unsigned residual of the differential equation. Now, noting that we want to train a neural network function $f(x|\theta)$, we can write:

$$
\dot{x} = \frac{d}{dt}f(x|\theta) \\
\ddot{x} = \frac{d^{2}}{dt^{2}}f(x|\theta)
$$

which we can presumably compute from the neural network.

If we want to fit some data with this constraint embedded in it, we can then just write our total loss as the mean squared error between the function and the data combined with the constraint loss:

$$
\mathcal{L} = \sum_{i} (x_{i}-f\left(t|\theta\right))^{2} + \mathcal{L_{\rm constrained}}
$$

So, instead of just finding a random function that fits the data, we are restricting the function "space" to only allow ones that satisfy the constraint equation. More precisely, we are considering only functions that come "close" to the constraint.

Now, the tricky part is doing the computation of the additional "constrained" loss term. This will require that we can take derivatives within the neural network. Before we get to that point, let's do a little setup.

To start with, we will do this experiment with some toy data generated using the exact solution to a damped harmonic oscillator:

$$
x(t) = A e^{-\kappa t} \cos(\omega t + \phi) \\
\kappa = \frac{\mu}{2m} \\
\omega = \sqrt{\omega_{0}^2-\kappa^2} \\
\omega_{0} = \sqrt{\frac{k}{m}}
$$


Note that this is only the solution for so-called "underdamped" motion where $\kappa \lt \omega_{0}$. The more general solution is more complex (literally, as it involves an analysis using complex numbers).

Be careful not to confuse the restoring parameter of the harmonic oscillator $k$ with the term dependent on the damping force $\kappa$. The quantities $\phi$ and $A$ are the phase and amplitude of the oscillations and are determined by the initial conditions. In the special case where the initial velocity is zero and $x_0$ is the initial position:

$$
\phi = \tan^{-1}\left(-\frac{\kappa}{\omega}\right) \\
A    = \frac{x_{0}}{\cos(\phi)}
$$


Let's go ahead and code that up. We'll also create a neural network with input and output layers and 3 hidden layers.


In [ ]:
#>>>RUN: L18.3-runcell01

def oscillator(d, w0, x):
    w = np.sqrt(w0**2-d**2)
    phi = np.arctan(-d/w)
    A = 1/(np.cos(phi))
    cos = torch.cos(phi+w*x)
    sin = torch.sin(phi+w*x)
    exp = torch.exp(-d*x)
    y  = exp*A*cos
    return y

class MLP(nn.Module):    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN):
        super().__init__()
        activation = nn.Tanh
        self.input = nn.Sequential(
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()
                      )
        self.hidden = nn.Sequential(
                    nn.Linear(N_HIDDEN, N_HIDDEN),
                    activation(),
                    nn.Linear(N_HIDDEN, N_HIDDEN),
                    activation(),
                    nn.Linear(N_HIDDEN, N_HIDDEN),
                    activation(),        
                    )

        self.output = nn.Linear(N_HIDDEN, N_OUTPUT)
        #self.fcs = nn.Sequential(*[nn.Linear(N_INPUT, N_HIDDEN),activation()])
        #self.fch = nn.Sequential(*[nn.Sequential(*[nn.Linear(N_HIDDEN, N_HIDDEN),activation()]) for _ in range(N_LAYERS-1)])
        #self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.input(x)
        x = self.hidden(x)
        x = self.output(x)
        return x
    


Now, what we can do is make some toy data using the exact damped harmonic oscillator solution. To do this, we will generate the data right in pytorch, which will make it easier to train and evaluate going forward. In this instance, we will generate a large dataset, but take only every 20th event to run our evaluation.

In [ ]:
#>>>RUN: L18.3-runcell02

d, w0 = 2, 20

# get the analytical solution over the full domain
x = torch.linspace(0,1,500).view(-1,1)
y = oscillator(d, w0, x).view(-1,1)

# slice out a small number of points from the LHS of the domain
x_data = x[0:200:20] # take just one event every 20th from 0 to 200
y_data = y[0:200:20]
print(x_data.shape, y_data.shape)

plt.figure()
plt.plot(x, y, label="Exact solution")
plt.scatter(x_data, y_data, color="tab:orange", label="Training data")
plt.xlabel('time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

Now, before we compute the full differential equation, let's go ahead and run a quick training to see how the performance looks if we just try to fit the first few points of this distribution without considering the constraint. For your reference, this is just like what we did with deep learning regression in the previous module of this course, 8.S50.2x.

In [ ]:
#>>>RUN: L18.3-runcell03

def plot_result(x,y,x_data,y_data,yh,fig,ax,images=[],xp=None):
    plt.cla()
    plt.plot(x,y, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
    plt.plot(x,yh, color="tab:blue", linewidth=4, alpha=0.8, label="Neural network prediction")
    plt.scatter(x_data, y_data[:,0], s=60, color="tab:orange", alpha=0.4, label='Training data')
    if xp is not None:
        plt.scatter(xp, -0*torch.ones_like(xp), s=60, color="tab:green", alpha=0.4, 
                    label='Physics loss training locations')
    l = plt.legend(loc=(0.865,0.60), frameon=False, fontsize="large")
    plt.setp(l.get_texts(), color="k")
    plt.xlim(-0.05, 1.05)
    plt.ylim(-1.1, 1.1)
    plt.text(0.865,0.7,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
    plt.axis("off")
    fig.canvas.draw() 
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)

    
# train standard neural network to fit training data
torch.manual_seed(123)
model = MLP(1,1,32)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
images = []
fig, ax = plt.subplots(figsize=(10,5))
y_data=torch.hstack((y_data,torch.ones(10,1)))
for i in range(1000):
    optimizer.zero_grad()
    yh = model(x_data)
    loss = torch.mean((yh[:,0:1]-y_data[:,0:1])**2)# use mean squared error
    loss.backward()
    optimizer.step()
    
    # plot the result as training progresses
    if (i+1) % 100 == 0: 
        print(loss.item())
        yh = model(x).detach()
        plot_result(x,y,x_data,y_data,yh,fig,ax,images)

imageio.mimsave('data/L18/training.gif', images, fps=10, loop=0)
Image(open('data/L18/training.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

As you can see, the neural network does a quite good job of fitting the training points, but fails miserably when extending beyond those points.

<a name='exercises_18_3'></a>     

| [Top](#section_18_0) | [Restart Section](#section_18_3) | [Next Section](#section_18_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 18.3.1</span>

Add 5 data points to the data set, corresponding to evenly spaced points sampled from the LAST 100 points, and re-run the neural network above. What is the last loss value that is printed? Report your new answer as a number with precision `1e-5`.

Does the NN do a better job at fitting when using additional data from the end of the dataset?

In [ ]:
#>>>EXERCISE: L18.3.1

#combine the original data set and last 5 evenly-spaced points
x_data = torch.vstack(#YOUR CODE HERE)
y_data = torch.vstack(#YOUR CODE HERE)

    
#RUN THE NN AGAIN
#YOUR CODE HERE
    

imageio.mimsave('data/L18/training2.gif', images, fps=10, loop=0)
Image(open('data/L18/training2.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory


<a name='section_18_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L18.4 Machine Learning the Pendulum Part II</h2>  

| [Top](#section_18_0) | [Previous Section](#section_18_3) | [Exercises](#exercises_18_4) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS18/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS18_vid4" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

Now, the fit is surpisingly good. In fact from the few points that we have fit so well, we can potentially start to glimmer how we can find the period, decay constant, and other parameters of the fit. However, this is actually not an obvious question, when you think about it.

*Can you figure out how we extract period, force, other fundamental parameters from the NN?* 

The answer to this might not be so satisfying. The only way we really know how to do this is by relying on our original knowledge of the differential equation. 

Recall from the pendulum that we have 

$$
\ddot{x(t)} = -\frac{k}{m} x(t) - \mu\dot{x}
$$

From the above equation, we have computed $x(t)$, this is just our neural network, which I remind you **is differentiable**. Likewise, we can compute this quickly with pytorch. Let's go ahead and do that and take a look at what we see. Specifically, we use the gradient of the $x$ array to find the velocity and then take the gradient of that to find the acceleration.




In [ ]:
#>>>RUN: L18.4-runcell01

#First let's apply the model
x_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)# sample locations over the problem domain
yhp = model(x_physics)
dx  = torch.autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]# computes dy/dx
dx2 = torch.autograd.grad(dx,  x_physics, torch.ones_like(dx),  create_graph=True)[0]# computes d^2y/dx^2
dx*=0.1
dx2*=0.01
plt.plot(x,y, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
plt.plot(x,yh, color="tab:blue", linewidth=4, alpha=0.8, label="Neural network prediction")
plt.scatter(x_physics.detach(), yhp.detach().numpy(), s=60, color="tab:orange", alpha=0.4, label='Training data')
plt.scatter(x_physics.detach(), dx.detach(), s=60, color="tab:green", alpha=0.4, label='0.1(dx/dt)')
plt.scatter(x_physics.detach(), dx2.detach(), s=60, color="tab:cyan", alpha=0.4, label='0.01(d$^{2}$x/dt$^{2}$)')
plt.scatter(x_physics.detach(), 0.1*dx2.detach()/yhp.detach(), s=60, color="blue", alpha=0.4, label='0.01(d$^{2}$x/dt$^{2}$)/x(t)')
plt.ylim(-3,3)
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("x(t)")
plt.show()

#note, your results may look slightly different than the related video,
#if you just retrained the NN in Exercise 18.3.1

Now, with the position, velocity, and acceleration, we can calculate a loss based on the differential equation constraint and then add this additional term to our loss. We will define two losses:

 * Mean Squared Error Loss (based on the difference between our fit function and the data)
 * Differential Equation Residual (based on the constraint)

As discussed previously, this gives:

$$
\mathcal{L} = \sum_{i} (x_{i}-f\left(t|\theta\right))^{2} + \mathcal{L_{\rm constrained}}\\
\mathcal{L_{\rm constrained}} = \left( m \ddot{x} + \mu\dot{x} +  kx \right)^2
$$

One important thing to realize is that we can calculate the mean squared error only for the data points we are using in the fit. However, since we have a functional form for $x(t)$, we can calculate $\mathcal{L_{\rm constrained}}$ over the entire range we are considering. Because of this difference, we need to be careful to match the scale of these two losses, otherwise the broader range used in calculating $\mathcal{L_{\rm constrained}}$ will cause it to dominate and the actual fitting to the data will be minimized. As you can see in the code below, we use a factor of $10^{-4}$ to accomplish this rescaling (this is inversely of order the number of bins, and there is some freedom in how we tune this).

To make our lives simple, we will first do this using the known values of $\mu$ and $\omega_{0}$. However, we will soon go beyond this.

One thing to note is that the neural network is pretty slow. It takes time to get this to converge.


In [ ]:
#>>>RUN: L18.4-runcell02

#x_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)# sample locations over the problem domain
mu, k = 2*d, w0**2
model = MLP(1,1,32)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

#Back to the origin data arrays that we were using before
x_data = x[0:200:20] # take just one event every 20th from 0 to 200
y_data = y[0:200:20]

torch.manual_seed(123)
images = []
fig, ax = plt.subplots(figsize=(10,5))
for i in range(20000):
    optimizer.zero_grad()
    
    # compute the "data loss"
    yh = model(x_data)
    loss1 = torch.mean((yh[:,0:1]-y_data[:,0:1])**2)# use mean squared error

    # compute the "physics loss"
    yhp = model(x_physics)
    dx  = torch.autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]# computes dy/dx
    dx2 = torch.autograd.grad(dx,  x_physics, torch.ones_like(dx),  create_graph=True)[0]# computes d^2y/dx^2
    physics = dx2 +  mu*dx +  k*yhp# computes the residual of the 1D harmonic oscillator differential equation
    loss2 = (1e-4)*torch.mean(physics**2)  #Note the multiplicative factor!
    
    # backpropagate joint loss
    loss = loss1 + loss2# add two loss terms together
    loss.backward()
    optimizer.step()
    
    
    # plot the result as training progresses
    if (i+1) % 100 == 0: 
        
        yh = model(x).detach()
        xp = x_physics.detach()

        print(loss.item())
        yh = model(x).detach()
        plot_result(x,y,x_data,y_data,yh,fig,ax,images)

from IPython.display import Image
a=imageio.mimsave('data/L18/training.gif', images, fps=10, loop=0)
Image(open('data/L18/training.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

I have to say I am starting to get amazed. With the addition of the differential equation constraint, our neural network is now predicting the entire range of the decaying exponential on top of a sinusoidal oscillation using just a knowledge of a few datapoints near the beginning. It's correctly extrapolating far away from what we have used as input in the training.

However, we have kind of cheated since we embedded in the loss a knowledge of the physics parameters of the system. Let's try to find these parameters too, so that we really learn everything!

The way we are going to do this is we are going to make two neural networks and run them simultaneously. We will train one for our prediction, and we will train one for our parameters. I know this sounds kind of crazy, but let's see how it goes. With the added complexity, this code takes even longer to run.

In [ ]:
#>>>RUN: L18.4-runcell03

#x_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)# sample locations over the problem domain
mu, k = 2*d, w0**2
model1 = MLP(1,1,32)
model2 = MLP(1,2,1)
#model2.Train = False

optimizer1 = torch.optim.Adam(model1.parameters(),lr=1e-4)
optimizer2 = torch.optim.Adam(model2.parameters(),lr=1e-2)

torch.manual_seed(123)
images = []
fig, ax = plt.subplots(figsize=(10,5))
for i in range(50000):
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    
    # compute the "data loss"
    yh = model1(x_data)
    loss1 = torch.mean((yh[:,0:1]-y_data[:,0:1])**2)# use mean squared error

    # compute the "physics loss"
    yhp   = model1(x_physics)
    decay = model2(x_physics)
    dx  = torch.autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]# computes dy/dx
    dx2 = torch.autograd.grad(dx,  x_physics, torch.ones_like(dx),  create_graph=True)[0]# computes d^2y/dx^2
    physics = dx2 +  torch.mean(decay[:,0:1])*dx +  torch.mean(decay[:,1:2])*yhp# computes the residual of the 1D harmonic oscillator differential equation
    loss2 = (1e-4)*torch.mean(physics**2)
    
    # backpropagate joint loss
    loss = loss1 + loss2# add two loss terms together
    loss.backward()
    optimizer1.step()
    optimizer2.step()
    
    
    # plot the result as training progresses
    if (i+1) % 100 == 0: 
        
        yh = model1(x).detach()
        xp = x_physics.detach()

        print(loss.item())
        yh = model1(x).detach()
        plot_result(x,y,x_data,y_data,yh,fig,ax,images)

from IPython.display import Image
a=imageio.mimsave('data/L18/training.gif', images, fps=10, loop=0)
Image(open('data/L18/training.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

Ok, now we need to see if the values make sense. Let's look at the output of our decay neural network and see what parameter values it has found.

In [ ]:
#>>>RUN: L18.4-runcell04

decays=model2(x_physics)
print("NN Mu:",decays[0][0].detach().numpy(),"Actual:",mu,"NN k: ",decays[0][1].detach().numpy(),"Actual:",k)

Note, in the video a smaller value (closer to 4) was computed. This is because the fit was run twice, and you will find the fit values get increasingly better with multiple runs.

So, we have pretty good numbers. Getting more accurate values of the decay constant and the spring constant $k$, would probably require additional datapoints. The differential equation constraint is only testing the validity of the fit function itself and therefore does almost nothing to determine parameters of the data itself.

I want to stress that we are giving the neural network 10 points and a constraint on the differential equation, and we are telling it to make a full prediction for the future. I don't know about you, but I am impressed. Just imagine trying to code up a much more complicated problems, it's not that hard.



<a name='exercises_18_4'></a>     

| [Top](#section_18_0) | [Restart Section](#section_18_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 18.4.1</span>

Again, add 5 data points to the data set, corresponding to evenly spaced points sampled from the LAST 100 points, and re-run the neural network above. How does this change the final value of the decay parameter? Report your new result for $\mu$, as a number with precision `1e-3`.

Before running your solution code, think a bit about what you expect will happen.

In [ ]:
#>>>EXERCISE: L18.4.1

#combine the original data set and last 5 evenly-spaced points
x_data = torch.vstack(#YOUR CODE HERE)
y_data = torch.vstack(#YOUR CODE HERE)

    
#RUN THE NN AGAIN
#YOUR CODE HERE
    
#print the parameters  
decays=model2(x_physics)
print("NN Mu:",decays[0][0].detach().numpy(),"Actual:",mu,"NN k: ",decays[0][1].detach().numpy(),"Actual:",k)